In [4]:
import pandas as pd

from preprocess import train_val_test_split, prepare_splitted_data


libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [5]:
PROJECT_PATH = '~/negative_feedback/'
DATA_PATH = f'{PROJECT_PATH}/data/ml20m/'

RELEVANCE_COL = 'rating'
RELEVANCE_THRESHOLD = 4

In [5]:
path_to_df = '~/negative_feedback/notebooks/ml20m/data/ratings.csv'
data = pd.read_csv(path_to_df)

In [6]:
data.rename(columns={"userId": "user_id", "movieId": "item_id"}, inplace=True)

In [7]:
data.duplicated(['user_id', 'item_id']).sum()

0

In [8]:
(data.groupby('item_id').count()['user_id'] < 5).mean()

0.3140517499252169

In [9]:
(data.groupby('user_id').count()['item_id'] < 5).mean()

0.0

In [12]:
train, val, test = train_val_test_split(data, RELEVANCE_THRESHOLD, RELEVANCE_COL, test_quantile=0.9)

Filtering items..
Number of items before 26744
Number of items after 18345
Interactions length before: 20000263
Interactions length after: 19984024
Filtering users..
Number of users before 138493
Number of users after 138493
Interactions length before: 19984024
Interactions length after: 19984024


In [16]:
val.user_id.nunique()

2480

In [17]:
train.user_id.nunique()

124969

In [18]:
def filter_last_positive(df):

    df = df.sort_values(['user_id', 'timestamp'])
    last_item = df.groupby('user_id').tail(1)
    last_item = last_item[last_item[RELEVANCE_COL] >= RELEVANCE_THRESHOLD]
    df = df[df['user_id'].isin(last_item['user_id'])]

    return df

In [19]:
val = filter_last_positive(val)

In [20]:
val.user_id.nunique()

1368

In [21]:
train.to_parquet(f'{DATA_PATH}/train.parquet', index=False)
test.to_parquet(f'{DATA_PATH}/test.parquet', index=False)
val.to_parquet(f'{DATA_PATH}/validation.parquet', index=False)